# ETL (Extrair, Transformar e Carregar) da camada Silver para Gold.

Este notebook realiza o processo de ETL para transformar e carregar os dados da camada Silver para a camada Gold no data lake. A camada Gold é otimizada para consultas analíticas e relatórios, garantindo que os dados estejam prontos para uso por ferramentas de BI e análise avançada.


## Colunas Mantidas para Dashboard

- **data_acidente_referencia**: Análise temporal dos acidentes (tendências, sazonalidade).
- **data_emissao_cat**: Avaliação de atrasos e eficiência no registro da CAT.
- **sexo**: Análises demográficas dos trabalhadores.
- **cbo**: Agrupamento por ocupação profissional.
- **cnae**: Análises por setor econômico do empregador.
- **tipo_acidente**: Classificação do tipo de ocorrência.
- **agente_causador_acidente**: Identificação das principais causas dos acidentes.
- **natureza_lesao**: Tipo de lesão decorrente do acidente.
- **parte_corpo_atingida**: Regiões do corpo mais afetadas.
- **cid10**: Classificação médica das lesões.
- **uf_acidente**, **municipio_acidente**: Localização geográfica do evento.
- **uf_empregador**, **municipio_empregador**: Localização do empregador.
- **qtd_acidentes**: Métrica principal para agregações e indicadores.

## Star Schema (Schema Estrela)

O Star Schema é um modelo de dados dimensional utilizado em data warehouses, no qual uma tabela fato central armazena métricas e chaves estrangeiras, enquanto tabelas dimensão ao seu redor armazenam atributos descritivos.

## Configuração do Ambiente de Desenvolvimento

In [ ]:
!pip install pyspark
!pip install psycopg2

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, count, when, avg, from_unixtime, unix_timestamp, lit, regexp_replace, to_date
from pyspark.sql.types import StringType

from psycopg2 import extras

import re
import psycopg2

import pandas as pd

### Conexão com o Banco de Dados e Extração do DataFrame Silver

In [ ]:
spark = SparkSession.builder \
    .appName("CSVtoPostgresETL") \
    .getOrCreate()

conn_params = {
    "host": "localhost",
    "database": "cat_db",
    "user": "admin",
    "password": "admin",
    "port": 5432
}

try:
    conn = psycopg2.connect(**conn_params)
    
    query = "SELECT * FROM ACIDENTE"
    
    pdf = pd.read_sql(query, conn)
    df_silver = spark.createDataFrame(pdf)
    
    print("Dados da camada Silver carregados")
    df_silver.show()

except Exception as e:
    print(f"Erro na leitura: {e}")
finally:
    if conn:
        conn.close()